In [1]:
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
from sklearn.datasets import load_iris
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from statsmodels.formula.api import ols
%load_ext autotime

In [2]:
df = pd.read_csv('data/kc_house_data.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21597 non-null  int64  
 1   date           21597 non-null  object 
 2   price          21597 non-null  float64
 3   bedrooms       21597 non-null  int64  
 4   bathrooms      21597 non-null  float64
 5   sqft_living    21597 non-null  int64  
 6   sqft_lot       21597 non-null  int64  
 7   floors         21597 non-null  float64
 8   waterfront     19221 non-null  float64
 9   view           21534 non-null  float64
 10  condition      21597 non-null  int64  
 11  grade          21597 non-null  int64  
 12  sqft_above     21597 non-null  int64  
 13  sqft_basement  21597 non-null  object 
 14  yr_built       21597 non-null  int64  
 15  yr_renovated   17755 non-null  float64
 16  zipcode        21597 non-null  int64  
 17  lat            21597 non-null  float64
 18  long  

In [4]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'kpmcdonough@gmail.com'
geopy.geocoders.options.default_timeout = 7
geolocator = Nominatim()
print(geolocator.headers)
print(geolocator.timeout)

{'User-Agent': 'kpmcdonough@gmail.com'}
7


In [5]:
locator = Nominatim()
coordinates = 47.7210, -122.319
location = locator.reverse(coordinates)
location.raw

{'place_id': 149083775,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 235228381,
 'lat': '47.72106685',
 'lon': '-122.31909342613385',
 'display_name': '837, Northeast 127th Street, Northgate, Seattle, King County, Washington, 98125, United States',
 'address': {'house_number': '837',
  'road': 'Northeast 127th Street',
  'suburb': 'Northgate',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98125',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.7210196', '47.7211141', '-122.3192057', '-122.3189812']}

In [6]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [7]:
test_df = df.head()

In [12]:
locator = Nominatim()

def return_neighborhood(location):
    if 'neighbourhood' in location['address'].keys():
        return location['address']['neighbourhood']
    elif 'suburb' in location['address'].keys():
        return location['address']['suburb']
    elif 'town' in location['address'].keys():
        return location['address']['town']
    elif 'hamlet' in location['address'].keys():
        return location['address']['hamlet']
    else:
        return "N/A"

test = []
for row in df.index:
    lat = df['lat'][row]
    long = df['long'][row]
    coords = lat,long
    location = locator.reverse(coords)
    location_dict = location.raw
    test.append(return_neighborhood(location_dict))

df


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,8,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,8,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,7,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,8,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287


In [13]:
test

['Rainier Beach',
 'Northgate',
 'Kenmore',
 'Fauntleroy',
 'Sammamish',
 'N/A',
 'Federal Way',
 'Des Moines Creek',
 'White Center',
 'Katesridge',
 'Robinswood',
 'Wedgwood',
 'Kenmore',
 'Sammamish',
 'Sunset Hill',
 'West Seattle',
 'Duvall',
 'Wallingford',
 'Rivers Bend',
 'Federal Way',
 'Licton Springs',
 'Mercer Island',
 'Lea Hill',
 'Kent',
 'White Horse Crossing',
 'Downtown',
 'Queen Anne',
 'Capitol Hill',
 'Wedgwood',
 'Redmond',
 'Issaquah',
 'Broadview',
 'Whittier Heights',
 'Whittier Heights',
 'Rock Creek',
 'Green Lake',
 'Redmond',
 'Phinney Ridge',
 'Auburn',
 'N/A',
 'Sammamish',
 'N/A',
 'Redmond Ridge East',
 'Queen Anne',
 'Renton',
 'Duvall',
 'Twin Lakes',
 'N/A',
 'Ravenna',
 'N/A',
 'North Hill',
 'Highlands Park',
 'Loyal Heights',
 'Crown Hill',
 'Laurelhurst',
 'N/A',
 'Covington',
 'Stonefield',
 'Sammamish Heights',
 'Summerwind',
 'N/A',
 'N/A',
 'N/A',
 'Green Lake',
 'Madison Valley',
 'Beacon Hill',
 'N/A',
 'Auburn',
 'Covington',
 'N/A',
 'N/A

In [14]:
test[0]

'Rainier Beach'

In [23]:
df['Neighborhood'] = 0
df

for row in df.index:
    df["Neighborhood"][row] = test[row]
df

<ipython-input-23-872bc9bb81f7>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Neighborhood"][row] = test[row]


,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,Neighborhood
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650,Rainier Beach
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639,Northgate
2,5631500400,2/25/2015,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062,Kenmore
3,2487200875,12/9/2014,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000,Fauntleroy
4,1954400510,2/18/2015,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503,Sammamish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,5/21/2014,360000.0,3,2.50,1530,1131,3.0,0.0,0.0,...,1530,0.0,2009,0.0,98103,47.6993,-122.346,1530,1509,Licton Springs
21593,6600060120,2/23/2015,400000.0,4,2.50,2310,5813,2.0,0.0,0.0,...,2310,0.0,2014,0.0,98146,47.5107,-122.362,1830,7200,White Center
21594,1523300141,6/23/2014,402101.0,2,0.75,1020,1350,2.0,0.0,0.0,...,1020,0.0,2009,0.0,98144,47.5944,-122.299,1020,2007,Leschi
21595,291310100,1/16/2015,400000.0,3,2.50,1600,2388,2.0,NaN,0.0,...,1600,0.0,2004,0.0,98027,47.5345,-122.069,1410,1287,Issaquah


In [36]:

top_neighborhoods = df['Neighborhood'].value_counts()[0:251].index
for row in df.index:
    if df['Neighborhood'][row] in top_neighborhoods:
        df['Neighborhood'][row] = df['Neighborhood'][row]
    else:
        df['Neighborhood'][row] = "Other"

<ipython-input-36-49cbccc07099>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Neighborhood'][row] = df['Neighborhood'][row]
<ipython-input-36-49cbccc07099>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Neighborhood'][row] = "Other"


In [39]:
df["Neighborhood"].value_counts()

N/A                 2235
Other               1069
Sammamish            886
Kirkland             701
Kent                 566
                    ... 
Eastwood Forest       12
Fernwood Estates      12
Endolyne              11
Wyngates              11
Willow Ridge          11
Name: Neighborhood, Length: 252, dtype: int64

In [43]:
for row in df.index:
    if df['Neighborhood'][row] == "N/A":
        df['Neighborhood'][row] = "Other"
df['Neighborhood'].value_counts()

Other              3304
Sammamish           886
Kirkland            701
Kent                566
Redmond             496
                   ... 
Glendale             12
Washington Park      12
Willow Ridge         11
Wyngates             11
Endolyne             11
Name: Neighborhood, Length: 251, dtype: int64

In [44]:
df.to_csv('data/KC_with_hoods')